In [2]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd
import rasterio as rio
from scipy.stats import gaussian_kde
import seaborn as sns

In [4]:

df = pd.read_csv( r'/Volumes/PhD/imagery/masters/output/hyper_compare/MvH_results.csv')
df


,BASIN_ID,RI,M1M,M1Md,M1PF,HM1M,HM1Md,HM1PF,HM1IM,HM1IMd,HM1IPF
0,12042,12.0,0.064260,0.053244,0.046768,0.070095,0.055214,0.047952,0.069559,0.052435,0.047878
1,12042,16.0,0.086179,0.067387,0.056811,0.096479,0.070677,0.058715,0.095525,0.066046,0.058596
2,12042,20.0,0.114659,0.084950,0.068855,0.131391,0.090059,0.071712,0.129830,0.082879,0.071532
3,12042,24.0,0.150996,0.106566,0.083226,0.176468,0.114104,0.087318,0.174084,0.103528,0.087061
4,12042,28.0,0.196295,0.132885,0.100274,0.232865,0.143554,0.105934,0.229444,0.128600,0.105577
...,...,...,...,...,...,...,...,...,...,...,...
371,22668,24.0,0.217452,0.275691,0.357919,0.254794,0.340418,0.417101,0.258005,0.354206,0.417102
372,22668,28.0,0.291314,0.372401,0.480807,0.343649,0.458428,0.553428,0.348095,0.476093,0.553430
373,22668,32.0,0.378143,0.480530,0.606062,0.444988,0.581294,0.682167,0.450542,0.600896,0.682169
374,22668,36.0,0.473558,0.590514,0.718774,0.551118,0.694841,0.788010,0.557361,0.713838,0.788011


In [11]:
basin = df[df['BASIN_ID']==17346]
basin_id = basin['BASIN_ID'].values[0]

basin

,BASIN_ID,RI,M1M,M1Md,M1PF,HM1M,HM1Md,HM1PF,HM1IM,HM1IMd,HM1IPF
72,17346,12.0,0.156104,0.182727,0.199330,0.176583,0.205386,0.232320,0.177556,0.214046,0.231946
73,17346,16.0,0.261148,0.312747,0.344347,0.300924,0.355728,0.405240,0.302801,0.371853,0.404567
74,17346,20.0,0.403113,0.480850,0.525609,0.463534,0.541170,0.605373,0.466303,0.562707,0.604538
75,17346,24.0,0.563404,0.653404,0.700363,0.634287,0.715870,0.775474,0.637379,0.736642,0.774744
76,17346,28.0,0.711460,0.793263,0.831393,0.776854,0.843312,0.886055,0.779544,0.858759,0.885561
77,17346,32.0,0.824911,0.886490,0.912299,0.874813,0.919981,0.945969,0.876750,0.929658,0.945682
78,17346,36.0,0.900023,0.940814,0.956418,0.933453,0.960876,0.975259,0.934686,0.966362,0.975107
79,17346,40.0,0.945059,0.970018,0.978856,0.965701,0.981296,0.988858,0.966431,0.984239,0.988781


In [ ]:
def find_crossing(x, y, threshold=0.5):
    for i in range(1, len(x)):
        if y[i-1] < threshold <= y[i]:
            x0, x1 = x[i-1], x[i]
            y0, y1 = y[i-1], y[i]
            return x0 + (threshold - y0) * (x1 - x0) / (y1 - y0)
    return np.nan 

In [8]:
RI = basin['RI'].values
RI

array([12., 16., 20., 24., 28., 32., 36., 40.])

In [9]:
model_columns = [col for col in basin.columns if col not in ['BASIN_ID', 'RI']]
model_columns

['M1M', 'M1Md', 'M1PF', 'HM1M', 'HM1Md', 'HM1PF', 'HM1IM', 'HM1IMd', 'HM1IPF']

In [12]:
results = {'BASIN_ID': [basin_id]} 
for col in model_columns:
    results[col] = [find_crossing(RI, basin[col].values)]


results_df = pd.DataFrame(results)
results_df

,BASIN_ID,M1M,M1Md,M1PF,HM1M,HM1Md,HM1PF,HM1IM,HM1IMd,HM1IPF
0,17346,22.417783,20.443921,19.434882,20.854247,19.111953,17.893938,20.787887,18.685763,17.908932
